Documentation

gym-examples/
  README.md
  setup.py
  gym_examples/
    __init__.py
    envs/
      __init__.py
      grid_world.py
    wrappers/
      __init__.py
      relative_position.py

Gym Documentation for Custom Env
`https://www.gymlibrary.dev/content/environment_creation/`

###Dependencies imports

In [ ]:
!pip install stable-baselines3 numpy

In [ ]:
!pip install gymnasium[mujoco] mujoco

In [ ]:
import gymnasium as gym
from stable_baselines3 import PPO
import imageio
import base64
from IPython.display import display, Image
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.logger import configure
import pandas as pd
import os
from gym import spaces
import pygame
from gym.envs.registration import register

import mujoco as mj
import numpy as np
import os

###Gyms Tutorial

In [ ]:
import gym
from gym import spaces
import pygame
import numpy as np


class GridWorldEnv(gym.Env):
    metadata = {"render_modes": ["human", "rgb_array"], "render_fps": 4}

    def __init__(self, render_mode=None, size=5):
        self.size = size  # The size of the square grid
        self.window_size = 512  # The size of the PyGame window

        # Observations are dictionaries with the agent's and the target's location.
        # Each location is encoded as an element of {0, ..., `size`}^2, i.e. MultiDiscrete([size, size]).
        self.observation_space = spaces.Dict(
            {
                "agent": spaces.Box(0, size - 1, shape=(2,), dtype=int),
                "target": spaces.Box(0, size - 1, shape=(2,), dtype=int),
            }
        )

        # We have 4 actions, corresponding to "right", "up", "left", "down"
        self.action_space = spaces.Discrete(4)

        """
        The following dictionary maps abstract actions from `self.action_space` to
        the direction we will walk in if that action is taken.
        I.e. 0 corresponds to "right", 1 to "up" etc.
        """
        self._action_to_direction = {
            0: np.array([1, 0]),
            1: np.array([0, 1]),
            2: np.array([-1, 0]),
            3: np.array([0, -1]),
        }

        assert render_mode is None or render_mode in self.metadata["render_modes"]
        self.render_mode = render_mode

        """
        If human-rendering is used, `self.window` will be a reference
        to the window that we draw to. `self.clock` will be a clock that is used
        to ensure that the environment is rendered at the correct framerate in
        human-mode. They will remain `None` until human-mode is used for the
        first time.
        """
        self.window = None
        self.clock = None

    def _get_obs(self):
        return {"agent": self._agent_location, "target": self._target_location}

    def _get_info(self):
        return {"distance": np.linalg.norm(self._agent_location - self._target_location, ord=1)}

    def reset(self, seed=None, options=None):
        # We need the following line to seed self.np_random
        super().reset(seed=seed)

        # Choose the agent's location uniformly at random
        self._agent_location = self.np_random.integers(0, self.size, size=2, dtype=int)

        # We will sample the target's location randomly until it does not coincide with the agent's location
        self._target_location = self._agent_location
        while np.array_equal(self._target_location, self._agent_location):
            self._target_location = self.np_random.integers(
                0, self.size, size=2, dtype=int
            )

        observation = self._get_obs()
        info = self._get_info()

        if self.render_mode == "human":
            self._render_frame()

        return observation, info

    def step(self, action):
        # Map the action (element of {0,1,2,3}) to the direction we walk in
        direction = self._action_to_direction[action]
        # We use `np.clip` to make sure we don't leave the grid
        self._agent_location = np.clip(
            self._agent_location + direction, 0, self.size - 1
        )
        # An episode is done iff the agent has reached the target
        terminated = np.array_equal(self._agent_location, self._target_location)
        reward = 1 if terminated else 0  # Binary sparse rewards
        observation = self._get_obs()
        info = self._get_info()

        if self.render_mode == "human":
            self._render_frame()

        return observation, reward, terminated, False, info

    def render(self):
        if self.render_mode == "rgb_array":
            #return 1
            return self._render_frame()

    def _render_frame(self):
        #return 1
        if self.window is None and self.render_mode == "human":
            pygame.init()
            pygame.display.init()
            self.window = pygame.display.set_mode((self.window_size, self.window_size))
        if self.clock is None and self.render_mode == "human":
            self.clock = pygame.time.Clock()

        canvas = pygame.Surface((self.window_size, self.window_size))
        canvas.fill((255, 255, 255))
        pix_square_size = (
            self.window_size / self.size
        )  # The size of a single grid square in pixels

        # First we draw the target
        pygame.draw.rect(
            canvas,
            (255, 0, 0),
            pygame.Rect(
                pix_square_size * self._target_location,
                (pix_square_size, pix_square_size),
            ),
        )
        # Now we draw the agent
        pygame.draw.circle(
            canvas,
            (0, 0, 255),
            (self._agent_location + 0.5) * pix_square_size,
            pix_square_size / 3,
        )

        # Finally, add some gridlines
        for x in range(self.size + 1):
            pygame.draw.line(
                canvas,
                0,
                (0, pix_square_size * x),
                (self.window_size, pix_square_size * x),
                width=3,
            )
            pygame.draw.line(
                canvas,
                0,
                (pix_square_size * x, 0),
                (pix_square_size * x, self.window_size),
                width=3,
            )

        if self.render_mode == "human":
            # The following line copies our drawings from `canvas` to the visible window
            self.window.blit(canvas, canvas.get_rect())
            pygame.event.pump()
            pygame.display.update()

            # We need to ensure that human-rendering occurs at the predefined framerate.
            # The following line will automatically add a delay to keep the framerate stable.
            self.clock.tick(self.metadata["render_fps"])
        else:  # rgb_array
            return np.transpose(
                np.array(pygame.surfarray.pixels3d(canvas)), axes=(1, 0, 2)
            )

    def close(self):
        if self.window is not None:
            pygame.display.quit()
            pygame.quit()

if __name__ == "__main__":
  register(
      id='gym_examples/GridWorld-v0',
      entry_point= '__main__:GridWorldEnv',
      max_episode_steps=300,
  )
  env = GridWorldEnv(render_mode="rgb_array")



/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.11/dist-packages/gym/envs/registration.py:542: UserWarning: WARN: Overriding environment gym_examples/GridWorld-v0
  logger.warn(f"Overriding environment {spec.id}")


In [ ]:
env.action_space

/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Discrete(4)

###humanoid env

In [ ]:
#taken from Gym API
class HumanoidCustomEnv(gym.Env):
    """Custom Humanoid environment using a MuJoCo XML model."""

    metadata = {"render_modes": ["human", "rgb_array"]}

    def __init__(self, xml_file, render_mode="rgb_array"):
        super().__init__()
        '''
        Need to get xml file in here instead
        # get xml model (from https://github.com/openai/mujoco-py#install-mujoco)
        mj_path = mujoco_py.utils.discover_mujoco()
        xml_path = os.path.join(mj_path, 'model', 'humanoid.xml')
        self.model = mujoco_py.load_model_from_path(xml_path)
        self.closesim = mujoco_py.MjSim(model)
        '''
        #this works better (after importing /common/ from mujuco's github repo)
        model = mj.MjModel.from_xml_path("/content/humanoid.xml")
        data = mj.MjData(model)

        # Define action and observation spaces like they are defined in Humanoid v5
        #actually, do we want to restrict things here?
        self.action_space = spaces.Box(low=-1, high=1, shape=(17,), dtype=np.float32) #maybe -.4 to .4
        #same?
        self.observation_space = spaces.Box(low=-np.inf, high=np.inf, shape=(376,), dtype=np.float32)

        #same as documentation
        self.viewer = None
        self.render_mode = render_mode

    def reset(self, seed=None, options=None):
        """Reset environment, place humanoid in a starting position."""
        self.sim.reset()

        # to do: Modify initial position (lying down not standing up)



    def step(self, action):
        """Advance simulation with given action. """


    #
    def is_done(self):
        """to be defined."""
        #we want different termination conditions than in humanoid v5

    #should be same as documentation
    def render(self):
        """Render simulation."""
        if self.render_mode == "rgb_array":
            #return 1
            return self._render_frame()

    def _render_frame(self):
        """Render simulation."""
        #follow documentation?

    def close(self):
        """Clean up resources."""
        if self.viewer is not None:
            self.viewer = None


register(
      id="HumanoidCustom-v0",
      entry_point="__main__:HumanoidCustomEnv",
      kwargs={"xml_file": "/content/simple.xml"}
  )
print(gym.envs.registry.keys())
env = HumanoidCustomEnv("/content/simple.xml", render_mode="rgb_array", )

dict_keys(['CartPole-v0', 'CartPole-v1', 'MountainCar-v0', 'MountainCarContinuous-v0', 'Pendulum-v1', 'Acrobot-v1', 'phys2d/CartPole-v0', 'phys2d/CartPole-v1', 'phys2d/Pendulum-v0', 'LunarLander-v3', 'LunarLanderContinuous-v3', 'BipedalWalker-v3', 'BipedalWalkerHardcore-v3', 'CarRacing-v3', 'Blackjack-v1', 'FrozenLake-v1', 'FrozenLake8x8-v1', 'CliffWalking-v0', 'Taxi-v3', 'tabular/Blackjack-v0', 'tabular/CliffWalking-v0', 'Reacher-v2', 'Reacher-v4', 'Reacher-v5', 'Pusher-v2', 'Pusher-v4', 'Pusher-v5', 'InvertedPendulum-v2', 'InvertedPendulum-v4', 'InvertedPendulum-v5', 'InvertedDoublePendulum-v2', 'InvertedDoublePendulum-v4', 'InvertedDoublePendulum-v5', 'HalfCheetah-v2', 'HalfCheetah-v3', 'HalfCheetah-v4', 'HalfCheetah-v5', 'Hopper-v2', 'Hopper-v3', 'Hopper-v4', 'Hopper-v5', 'Swimmer-v2', 'Swimmer-v3', 'Swimmer-v4', 'Swimmer-v5', 'Walker2d-v2', 'Walker2d-v3', 'Walker2d-v4', 'Walker2d-v5', 'Ant-v2', 'Ant-v3', 'Ant-v4', 'Ant-v5', 'Humanoid-v2', 'Humanoid-v3', 'Humanoid-v4', 'Humanoid-v5

/usr/local/lib/python3.11/dist-packages/gym/envs/registration.py:542: UserWarning: WARN: Overriding environment HumanoidCustom-v0
  logger.warn(f"Overriding environment {spec.id}")


In [ ]:
env = HumanoidCustomEnv("/content/humanoid.xml", render_mode="rgb_array", )


/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
env.action_space

/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Box(-1.0, 1.0, (17,), float32)